In [1]:
import csv
import pandas as pd
import os
import sys
from sklearn.metrics import pairwise_distances

In [2]:
class CosineSimilarity():
    def __init__(self,*args, **kwargs):
        self.test = args[1]
        self.train = args[0] 
        
        
    
    def calculate_cosine(self):
        cosine_sim = 1-pairwise_distances(self.train,self.test, metric="cosine")
        result= pd.DataFrame(cosine_sim)
        rec_index= self.filter(result)
        return rec_index
        
    def filter(self,result):
        result= result.stack()
        result= result.sort_values(ascending=False )
        result= result.iloc[0:7:1]
        rec_index= []
        for (index,value) in result.iteritems():
            rec_index.append(((index[0]+1),value))
        return rec_index


In [3]:
class preProcessor():
    def __init__(self, *args, **kwargs):
        pass
    
    def columnSelector (self,traindata,testdata):
        matching = set(traindata) - set(testdata)
        remove= list(matching)
        traindata.drop(remove,axis=1, inplace=True)
        order = list(traindata)
        testdata = testdata[order]
        return traindata,testdata
    
    def usertouser(self):
        pass

    def usertocompany(self):
        pass

    def companytocompany(self):
        pass

In [4]:
class CollaborativeRec():
    def __init__(self, *args, **kwargs):
        self.userdata = args[0]
        # Information of jobs : key = jobId
        self.jobdata= args[1]
        self.test= args[2]
        #jobMapper must be a (userId,jobId)
        self.jobMapper= self.userdata['Jobid']
        self.preProcessor = preProcessor()
    
    # function that ineract with Recommend.py
    def getRecord(self):  
        joblist1=self.user2user()
        joblist2=self.user2company()
        joblist = joblist1 + joblist2
        for each in joblist:
            print(each)
        #filter the list
        
    def user2user(self):
        self.similar= []
        #need to slice userdata according to the test data
        self.userdata, self.test= self.preProcessor.columnSelector(self.userdata, self.test)
        #get user similarity data
        self.cos= CosineSimilarity(self.userdata,self.test)
        self.similar = self.cos.calculate_cosine()
        jobList= []
        jobjson= {}
        for each in self.similar:
            #find the job Id from user Id
            jobId= self.jobMapper[each[0]]
            #find job in self.job
            job = self.jobdata.loc[jobId]
            #put job, value in dict
            jobdict= job.to_dict()
            jobdict['Jobid']= jobId
            jobjsontemp = {'jobdata':jobdict,'similarity':each[1]}
            jobList.append(jobjsontemp)
        return jobList
            
    #item to item based
    def user2company(self):
        self.similar= []
        jobdata= self.jobdata.copy()
        testdata=  self.test.copy() 
        self.jobdata_train, self.test_train= self.preProcessor.columnSelector(jobdata,testdata)
        #get user to company similarity data
        self.cos= CosineSimilarity(self.jobdata_train,self.test_train)
        self.similar = self.cos.calculate_cosine()
        jobList=[]
        jobjson={}
        for each in self.similar:
            job = self.jobdata.loc[each[0]]
            jobdict= job.to_dict()
            jobdict['Jobid']= each[0]
            jobjsontemp = {'jobdata':jobdict,'similarity':each[1]}
            jobList.append(jobjsontemp)
        return jobList
           

In [5]:
class Recommend():
    def __init__(self, *args, **kwargs):
        self.test= args[0]
        self.csvReader()
        self.collab= CollaborativeRec(self.userdata,self.jobdata, self.test) 
        #self.content= ContentRec(self.jobdata,self.test)
    def readData(self,id):
        #Read test,user,job json data or csv data
        #call collab
        #return json
        pass

    def getData(self):
        data = self.collab.getRecord()
        return data


    def csvReader(self):
        #Define relative path for csv files
        self.userdata = pd.read_csv('src/data/finalUser.csv')
        self.userdata.set_index('UserId',inplace = True)
        self.jobdata = pd.read_csv('src/data/finalJob.csv')
        self.jobdata.set_index('Jobid',inplace = True)


In [6]:
if __name__== '__main__':
    testdata = {'php': 1, 'python': 1, 'qualification': 2, 'xperience': 5,'level': 2, 'age': 25}
    frame = pd.DataFrame.from_dict([testdata])
    a= Recommend(testdata)
    print(a)

FileNotFoundError: File b'src/data/finalUser.csv' does not exist